# I'll try to use our DCGAN model on a dataset of planes to see if we can make it generate soething that looks close to a plane

In [66]:
from torch.nn import ConvTranspose2d
from torch.nn import BatchNorm2d
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import LeakyReLU
from torch.nn import ReLU
from torch.nn import Tanh
from torch.nn import Sigmoid
from torch import flatten
from torch import nn

class Generator(nn.Module):
    def __init__(self, inputDim=100, outputChannels=3):
        super(Generator, self).__init__()
        self.ct1 = ConvTranspose2d(in_channels=inputDim,
            out_channels=128, kernel_size=4, stride=1, padding=0, bias=False)  # -> (128, 4, 4)
        self.relu1 = ReLU()
        self.batchNorm1 = BatchNorm2d(128)
        
        self.ct2 = ConvTranspose2d(in_channels=128, out_channels=64,
            kernel_size=4, stride=2, padding=1, bias=False)  # -> (64, 8, 8)
        self.relu2 = ReLU()
        self.batchNorm2 = BatchNorm2d(64)
        
        self.ct3 = ConvTranspose2d(in_channels=64, out_channels=32,
            kernel_size=4, stride=2, padding=1, bias=False)  # -> (32, 16, 16)
        self.relu3 = ReLU()
        self.batchNorm3 = BatchNorm2d(32)
        
        self.ct4 = ConvTranspose2d(in_channels=32,
            out_channels=outputChannels, kernel_size=4, stride=2,
            padding=1, bias=False)  # -> (3, 32, 32)
        self.tanh = Tanh()

    def forward(self, x):
        x = self.ct1(x)
        x = self.relu1(x)
        x = self.batchNorm1(x)

        x = self.ct2(x)
        x = self.relu2(x)
        x = self.batchNorm2(x)

        x = self.ct3(x)
        x = self.relu3(x)
        x = self.batchNorm3(x)

        x = self.ct4(x)
        output = self.tanh(x)
        return output

In [67]:
class Discriminator(nn.Module):
	def __init__(self, depth, alpha=0.2):
		super(Discriminator, self).__init__()
		
		self.conv1 = Conv2d(in_channels=depth, out_channels=32,
				kernel_size=4, stride=2, padding=1)
		self.leakyRelu1 = LeakyReLU(alpha, inplace=True)
		
		self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=4,
				stride=2, padding=1)
		self.leakyRelu2 = LeakyReLU(alpha, inplace=True)
		
		self.fc1 = Linear(in_features=4096, out_features=512)
		self.leakyRelu3 = LeakyReLU(alpha, inplace=True)
		
		self.fc2 = Linear(in_features=512, out_features=1)
		self.sigmoid = Sigmoid()
		
	def forward(self, x):
		x = self.conv1(x)
		x = self.leakyRelu1(x)
		
		x = self.conv2(x)
		x = self.leakyRelu2(x)
		
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.leakyRelu3(x)
		
		x = self.fc2(x)
		output = self.sigmoid(x)
		return output

In [57]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision import transforms
from sklearn.utils import shuffle
from imutils import build_montages
from torch.optim import Adam
from torch.nn import BCELoss
from torch import nn
import numpy as np
import argparse
import torch
import cv2
import os

def weights_init(model):
	classname = model.__class__.__name__
	
	if classname.find("Conv") != -1:
		nn.init.normal_(model.weight.data, 0.0, 0.02)
	
	elif classname.find("BatchNorm") != -1:
		nn.init.normal_(model.weight.data, 1.0, 0.02)
		nn.init.constant_(model.bias.data, 0)

##### Let's get setup a Dataloder

In [ ]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

from torchvision import transforms
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transforms.ToTensor())

import torch
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

transformed_cifar10 = datasets.CIFAR10(
        data_path, train=True, download=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean,
                std)
        ]))

transformed_cifar10_test = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

label_map = {0: 0}
class_names = ['airplane']
cifar2 = [(img, label_map[label]) for img, label in transformed_cifar10
    if label == 0]
cifar2_val = [(img, label_map[label]) for img, label in transformed_cifar10_test
    if label == 0]

Files already downloaded and verified
Files already downloaded and verified


In [80]:
NUM_EPOCHS = 500
BATCH_SIZE = 128

In [78]:
len(cifar2)

5000

In [82]:
val_loader = DataLoader(cifar2, batch_size=64, shuffle=False)

In [86]:
stepsPerEpoch = len(val_loader.dataset) // BATCH_SIZE

print("[INFO] building generator...")
gen = Generator(inputDim=100, outputChannels=3)
gen.apply(weights_init)

print("[INFO] building discriminator...")
disc = Discriminator(depth=3)
disc.apply(weights_init)

# initialize optimizer for both generator and discriminator
genOpt = Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999),
	weight_decay=0.002 / NUM_EPOCHS)
discOpt = Adam(disc.parameters(), lr=0.0002, betas=(0.5, 0.999),
	weight_decay=0.002 / NUM_EPOCHS)

criterion = BCELoss()

[INFO] building generator...
[INFO] building discriminator...


### Now the Training loop

In [ ]:
print("[INFO] starting training...")
benchmarkNoise = torch.randn(256, 100, 1, 1)

realLabel = 1
fakeLabel = 0

for epoch in range(NUM_EPOCHS):
	print("[INFO] starting epoch {} of {}...".format(epoch + 1,
		NUM_EPOCHS))
	
	epochLossG = 0
	epochLossD = 0
	for x in val_loader:
		disc.zero_grad()
		
		images = x[0]
		
		bs =  images.size(0)
		labels = torch.full((bs,), realLabel, dtype=torch.float)
		
		output = disc(images).view(-1)
		errorReal = criterion(output, labels)
		errorReal.backward()

		noise = torch.randn(bs, 100, 1, 1)
		fake = gen(noise)
		labels.fill_(fakeLabel)

		output = disc(fake.detach()).view(-1)
		errorFake = criterion(output, labels)
		errorFake.backward()
		errorD = errorReal + errorFake
		discOpt.step()

		gen.zero_grad()
		labels.fill_(realLabel)
		output = disc(fake).view(-1)
		errorG = criterion(output, labels)
		errorG.backward()
		genOpt.step()
		epochLossD += errorD
		epochLossG += errorG

	print("[INFO] Generator Loss: {:.4f}, Discriminator Loss: {:.4f}".format(
		epochLossG / stepsPerEpoch, epochLossD / stepsPerEpoch))
	
	if (epoch + 1) % 5 == 0:
		# set the generator in evaluation phase, make predictions on
		# the benchmark noise, scale it back to the range [0, 255],
		# and generate the montage
		gen.eval()
		images = gen(benchmarkNoise)
		images = images.detach().cpu().numpy().transpose((0, 2, 3, 1))
		images = ((images * 127.5) + 127.5).astype("uint8")
		vis = build_montages(images, (64, 64), (16, 16))[0]
		vis = cv2.cvtColor(vis, cv2.COLOR_RGB2BGR)
		
		# save output
		p = os.path.join('output_planes', "epoch_{}.png".format(
			str(epoch + 1).zfill(4)))
		cv2.imwrite(p, vis)
		gen.train()

[INFO] starting training...
[INFO] starting epoch 1 of 500...
[INFO] Generator Loss: 4.6239, Discriminator Loss: 0.9458
[INFO] starting epoch 2 of 500...
[INFO] Generator Loss: 5.0220, Discriminator Loss: 1.0433
[INFO] starting epoch 3 of 500...
[INFO] Generator Loss: 4.0225, Discriminator Loss: 1.1888
[INFO] starting epoch 4 of 500...
[INFO] Generator Loss: 4.0761, Discriminator Loss: 1.0474
[INFO] starting epoch 5 of 500...
[INFO] Generator Loss: 4.2128, Discriminator Loss: 1.0556
[INFO] starting epoch 6 of 500...
[INFO] Generator Loss: 4.1319, Discriminator Loss: 1.0278
[INFO] starting epoch 7 of 500...
[INFO] Generator Loss: 4.0299, Discriminator Loss: 0.9950
[INFO] starting epoch 8 of 500...
[INFO] Generator Loss: 4.3279, Discriminator Loss: 0.9153
[INFO] starting epoch 9 of 500...
[INFO] Generator Loss: 4.4091, Discriminator Loss: 0.8994
[INFO] starting epoch 10 of 500...
[INFO] Generator Loss: 4.5421, Discriminator Loss: 0.8574
[INFO] starting epoch 11 of 500...
[INFO] Generator

In [ ]:
torch.save(gen.state_dict(), './models/gen.pt')
torch.save(disc.state_dict(), './models/disc.pt')